In [1]:
# -*- coding: utf-8 -*-

In [2]:
import re
import os
import sys
import pandas as pd
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import json
from scipy import stats
import csv
import pprint
from collections import defaultdict
import time
import datetime
import math
import glob
import praw
# import matplotlib.pyplot as plt; plt.rcdefaults()
# from IPython.display import Markdown, display

In [3]:
pd.options.display.max_rows = 1999

In [4]:
QUARANTINE_DATE = "28/09/2018"
quarDateUnix = time.mktime(datetime.datetime.strptime(QUARANTINE_DATE, "%d/%m/%Y").timetuple())

In [5]:
ALL_COMMENTS_FILE = "/mnt/storage/quarantine/data/TheRedPill-comments.csv"   
ALL_SUBMISSIONS_FILE = "/mnt/storage/quarantine/data/TheRedPill-submissions.csv" 
NEW_USER_POSTS_FILE = "/mnt/storage/quarantine/data/newUserPosts.csv" 
TOTAL_USERS = "/mnt/storage/quarantine/data/candidate_control_subs_unique_users.csv"

In [6]:
df_comments = pd.read_csv(ALL_COMMENTS_FILE,
                          usecols = ["created_utc", "subreddit", "author", "score"])
df_submissions = pd.read_csv(ALL_SUBMISSIONS_FILE,
                      usecols = ["created_utc", "subreddit", "author", "score"])
df = pd.concat([df_comments, df_submissions], ignore_index=True)

/nethome/sjhaver3/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


In [7]:
dfPast = df[df.created_utc <= quarDateUnix]
dfFuture = df[df.created_utc > quarDateUnix]

dfPastAuthors = set(dfPast['author'])
dfFutureAuthors = set(dfFuture['author'])

In [8]:
len(dfPastAuthors)

19369

In [9]:
len(dfFutureAuthors)

8511

In [10]:
newAuthors = dfFutureAuthors - dfPastAuthors
len(newAuthors)

4708

In [11]:
quot = "\""
text = ""
for author in newAuthors:
    text = text + quot + author + quot + ","
lb = "("
rb = ")"
print (lb + text[:-1] + rb)    

("drewdrewadraw","ConsistentRelease","Playstation_5","Standgrounding","krakarot","Ketonian_Empire","TheEarlobe","SwoleBenji","acoolguy12334","Nevesj98G","sturbine","Jack-Casper","Kiroboss","LouderWithBowser","MMorte","her_958_resistors","mahin300","youshouldjotthatdown","AwareVehicle","melange_merchant","rbillyvee","LordTilde","calibroni16000","viensviens","Granite_Pill","AnyGoal","Kruppstahlterminator","surrealismen","RakishRed","CJ090","LoveAtlas","NoahHaders","urbanfoh","DooleyBoy1836","MinimalistLifestyle","djputinvodkaparrot","wine_vs_milk","RoadToZion1119","randomtextleafs","_Legendairy_","DeFazHS","gikere","NoGotGits","Mazarati69","IvankasBabyDaddy","shasha2003","PoliticallyCo_rekt","theaceshinigami","Builtblind","Lucifers_sausage","shanghaiboii","Senser69","neos_trenchcoat","ShadowKnight324","blissfulrage","VeryCatholicGoodBoy","Belt321","dgrawrpla","triplewitching2","Groovez83","adonisosiris","fag_bucket","WarrenLecter","Cactusinthetrap","GreenBarbers","redditinoid","zettaflar

## We used BigQuery at this stage to Retrieve the User Posts Before the Quarantining

In [12]:
newDf = pd.read_csv(NEW_USER_POSTS_FILE)

In [13]:
newDfSubs = newDf.groupby('subreddit').size().reset_index(name='post_counts')

In [14]:
g = newDfSubs.sort_values(by=['post_counts'], ascending=False)[:100]
g

,subreddit,post_counts
440,AskReddit,33981
4873,The_Donald,28985
3959,PurplePillDebate,25097
3024,MGTOW,21927
795,Braincels,14497
1193,CringeAnarchy,9335
10528,worldnews,8364
8262,milliondollarextreme,8323
8853,politics,7950
10253,unpopularopinion,6315


In [15]:
k = newDf.groupby(['subreddit', 'author']).size().to_frame(name = 'size').reset_index()

In [17]:
kUsers = k.groupby('subreddit').size().reset_index(name='TRP_author_counts')
h = kUsers.sort_values(by=['TRP_author_counts'], ascending=False)[:100]
h

,subreddit,TRP_author_counts
440,AskReddit,1106
10528,worldnews,549
8796,pics,507
9908,todayilearned,487
7200,funny,481
4873,The_Donald,429
8469,news,417
7244,gaming,416
5750,asktrp,400
4397,Showerthoughts,392


In [18]:
TOTAL_df = pd.read_csv(TOTAL_USERS)
TOTAL_df.rename(columns = {'Subreddit':'subreddit'}, inplace = True)

In [19]:
TOTAL_df.columns

Index(['subreddit', 'Total_users'], dtype='object')

In [20]:
kUsers = kUsers.join(TOTAL_df.set_index('subreddit'), on='subreddit')

In [21]:
kUsers['TRP_users/Total_users'] = kUsers['TRP_author_counts']/kUsers['Total_users']

In [22]:
g = kUsers[kUsers.Total_users >= 100].sort_values(by=['TRP_users/Total_users'], ascending=False)[:100]
g

,subreddit,TRP_author_counts,Total_users,TRP_users/Total_users
5750,asktrp,400,11434.0,0.034983
607,BattleOfTheSexes,13,413.0,0.031477
718,BlackPillScience,10,344.0,0.029070
8135,marriedredpill,32,1229.0,0.026037
3959,PurplePillDebate,89,3524.0,0.025255
3025,MGTOW2,19,758.0,0.025066
5336,WhereAreAllTheGoodMen,84,3453.0,0.024327
5715,askMRP,24,1095.0,0.021918
4794,The30DayChallenge,3,149.0,0.020134
4005,RPChristians,6,310.0,0.019355


In [23]:
g = kUsers.sort_values(by=['TRP_users/Total_users'], ascending=False)[:100]
g

,subreddit,TRP_author_counts,Total_users,TRP_users/Total_users
5750,asktrp,400,11434.0,0.034983
607,BattleOfTheSexes,13,413.0,0.031477
718,BlackPillScience,10,344.0,0.029070
8135,marriedredpill,32,1229.0,0.026037
3959,PurplePillDebate,89,3524.0,0.025255
3025,MGTOW2,19,758.0,0.025066
5336,WhereAreAllTheGoodMen,84,3453.0,0.024327
5715,askMRP,24,1095.0,0.021918
4794,The30DayChallenge,3,149.0,0.020134
4005,RPChristians,6,310.0,0.019355
